### Mount gdrive

### This cell is for Google colaboratory users.

### Parameter setting

In [1]:
class_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
firstLayerSize = 512
secondLayerSize = 512
outputNeuron = 10

outputDir = "./models/"
epochs = 30
batch_size = 32
model_name = outputDir + "firstLayerSize"+str(firstLayerSize)+"secondLayerSize"+str(secondLayerSize)+"outputNeuron"+str(outputNeuron)+ "class" + str(class_list) + ".h5"

### Import libraries

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
%matplotlib inline

### Load CIFAR10 data set

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Extract data in class_list
train_filter = np.where( [ (element in class_list) for element in y_train] )
test_filter = np.where( [ (element in class_list) for element in y_test] )
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

In [5]:
# Normalize image data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [6]:
# Convert vectors to binary class
y_train = keras.utils.to_categorical(y_train, outputNeuron)
y_test = keras.utils.to_categorical(y_test, outputNeuron)

### Make model

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(firstLayerSize, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(secondLayerSize, activation = 'relu'))
model.add(Dense(outputNeuron, activation = 'softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

### Train model

In [9]:
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
fit = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
1563/1563 [==============================] - 302s 192ms/step - loss: 1.7591 - accuracy: 0.3473 - val_loss: 1.4519 - val_accuracy: 0.4676
Epoch 2/30
1563/1563 [==============================] - 288s 184ms/step - loss: 1.4347 - accuracy: 0.4805 - val_loss: 1.3061 - val_accuracy: 0.5288
Epoch 3/30
1563/1563 [==============================] - 260s 166ms/step - loss: 1.3066 - accuracy: 0.5296 - val_loss: 1.1746 - val_accuracy: 0.5771
Epoch 4/30
1563/1563 [==============================] - 324s 207ms/step - loss: 1.2093 - accuracy: 0.5672 - val_loss: 1.0916 - val_accuracy: 0.6096
Epoch 5/30
1563/1563 [==============================] - 287s 183ms/step - loss: 1.1313 - accuracy: 0.5988 - val_loss: 1.0314 - val_accuracy: 0.6361
Epoch 6/30
1563/1563 [==============================] - 277s 177ms/step - loss: 1.0676 - accuracy: 0.6217 - val_loss: 0.9868 - val_accuracy: 0.6472
Epoch 7/30
1563/1563 [==============================] - 275s 176ms/step - loss: 1.0068 - accuracy: 0.6436 - val_

In [10]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

313/313 [==============================] - 12s 38ms/step - loss: 0.6477 - accuracy: 0.7879
Test loss: 0.6476595997810364
Test accuracy: 0.7878999710083008


### Save model

In [11]:
model.save(model_name, include_optimizer = False)